In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cd C:\Users\Amy\Google Drive\CarVi\CarviCSV

C:\Users\Amy\Google Drive\CarVi\CarviCSV


#PERIOD

In [3]:
periodsDF=pd.read_csv('periods.csv')
#period.head()
periodsDF.shape
periodsDF.columns

Index([u'period_id', u'camera_id', u'mileage', u'collision_mileage', u'open'], dtype='object')

In [4]:
periodsDF['period_id']=periodsDF['period_id'].apply(str) # changing the period_ID to string

In [5]:
#periodsDF['camera_id'].unique()
#print len(periods[periods['mileage']==0])
#periods[periods['mileage']==0].describe(include='all')
#periods[periods['mileage']>0].describe(include='all')
#periods.head(40)

#MOTION

In [8]:
motionsDF=pd.read_csv('motions.csv')
#motionsDF.head()
motionsDF['motion_id']=motionsDF['motion_id'].apply(str)
motionsDF['period_id']=motionsDF['period_id'].apply(str)
motionsDF['event']=motionsDF['event'].apply(str)
motionsDF['time_stamp']=pd.to_datetime(motionsDF['time_stamp'])
motionsDF.shape

(64893, 8)

In [9]:
#motions.describe(include='all')
#motions['period_id'].value_counts()

In [10]:
#motionsDF['event'].value_counts().plot(kind='bar')
# 4=lane departure, 7=sudden acceleration, 1=lane change, 5=forward collision
#print motionsDF['event'].value_counts()'''

#MILES

In [11]:
milesDF=pd.read_csv('miles.csv')
milesDF['mile_id']=milesDF['mile_id'].apply(str)
milesDF['period_id']=milesDF['period_id'].apply(str)
milesDF['motion_id']=milesDF['motion_id'].apply(str)
#miles.describe(include='all')
#milesDF.head()
#milesDF.describe(include='all')
milesDF.shape


(887, 5)

In [12]:
#pd.merge(motions ,miles[miles['period_id']=='40'],  how='inner', sort='mile_id')
#miles[miles['mileage']==0].describe(include='all')

#COLLISION

In [13]:
collisionsDF=pd.read_csv('collision.csv')
#collision.head

collisionsDF['collision_id']=collisionsDF['collision_id'].apply(str)
collisionsDF['motion_id']=collisionsDF['motion_id'].apply(str)
collisionsDF['reaction']=collisionsDF['reaction'].apply(str)
collisionsDF.shape


(6535, 4)

In [14]:
#collisionsDF.describe(include='all')
#collision['reaction'].value_counts().plot(kind='bar')

#FRONT

In [15]:
frontsDF=pd.read_csv('front.csv')
frontsDF['front_id']=frontsDF['front_id'].apply(str)
frontsDF['motion_id']=frontsDF['motion_id'].apply(str)
#frontsDF.head()
frontsDF.shape

(22245, 4)

In [16]:
#frontsDF.describe(include='all')

#DEPARTURE

In [17]:
departuresDF=pd.read_csv('departure.csv')
#departuresDF.head()
departuresDF['departure_id']=departuresDF['departure_id'].apply(str)
departuresDF['motion_id']=departuresDF['motion_id'].apply(str)
departuresDF['lane_kind']=departuresDF['lane_kind'].apply(str)
departuresDF['reaction']=departuresDF['reaction'].apply(str)
departuresDF['direction']=departuresDF['direction'].apply(str)
departuresDF.shape

(25176, 5)

In [18]:
#departure.describe(include='all')
#departure['direction'].value_counts()

#SUDDENS

In [20]:
suddensDF=pd.read_csv('suddens.csv')
suddensDF['sudden_id']=suddensDF['sudden_id'].apply(str)
suddensDF['motion_id']=suddensDF['motion_id'].apply(str)
suddensDF['situation']=suddensDF['situation'].apply(str)
suddensDF['front_reason']=suddensDF['front_reason'].apply(str)
#suddensDF.head()
suddensDF.shape

(766, 4)

In [21]:
#suddensDF.describe(include='all')
#suddens['situation'].value_counts()
# 1=sudden stop, 3=suddent acceleration
#suddens['front_reason'].value_counts()

In [22]:
#pd.merge(motions, suddens[suddens['situation']=='1.0'], how='inner', on='motion_id')
#pd.merge(motions[motions['period_id']=='2124'], suddens, how='left', on='motion_id')

In [23]:
uniqueperiods100=motionsDF['period_id'].unique()[:100]
len(uniqueperiods100)

100

In [28]:
np.mean(motionsDF[motionsDF['period_id']=='40']['bias'])

5.371428571428571

# Building the Data Frame 

In [32]:
bigframe=pd.DataFrame()
motionsDF=motionsDF[motionsDF.event!='100']# when does an actual driving trip start and end? why do some period_id have 100 vs 900, etc
motionsDF=motionsDF[motionsDF.event!='900']
eventlist=['normalE','passingE','changeE','departureE','collisionE','suddenE','frontE']
eventd={'1':'normalE','2':'passingE','3':'changeE','4':'departureE','5':'collisionE','6':'suddenE','7':'frontE'}

suddenslist=['suddenStop','suddenBrake','suddenAccel']
suddensd={'1.0':'suddenStop', '2.0':'suddenBrake', '3.0':'suddenAccel', 'nan':'suddenNaN'}

colReactlist=['CReact_ignore_departL','CReact_reduceSpeed','CReact_changeL']
colReactd={'1.0':'CReact_ignore_departL','2.0':'CReact_reduceSpeed', '3.0':'CReact_changeL','nan':'colReactNaN'}

motionsDF['bias']=motionsDF['bias']+5

for period in uniqueperiods100:
    periodframe=pd.DataFrame()
    periodframe=periodframe.append({'period_id':period},ignore_index=True)
    
    #total miles
    periodframe['total_mileage']=float(periodsDF[periodsDF['period_id']==period]['mileage'])
    
    #event counts
    periodevents=motionsDF[motionsDF['period_id']==period]['event'].value_counts().to_frame().reset_index()
    periodevents['index']=periodevents['index'].map(eventd)
    for event in eventlist:
         if event not in np.array(periodevents['index']):
                periodframe[event]=0
    periodevents=periodevents.set_index('index')
    periodeventsT=periodevents.T
    periodframe=pd.concat([periodframe,periodeventsT], axis=1)
    
    # sudden counts
    motionids=motionsDF[(motionsDF['event']=='6')&(motionsDF['period_id']==period)]
    motionsudden=pd.merge(motionids,suddensDF, how='inner', on='motion_id')['situation'].value_counts().dropna().to_frame().reset_index()
    motionsudden['index']=motionsudden['index'].map(suddensd)
    for sEvent in suddenslist:
         if sEvent not in np.array(motionsudden['index']):
                periodframe[sEvent]=0
    motionsudden=motionsudden.set_index('index')
    motionsuddenT=motionsudden.T
    periodframe=pd.concat([periodframe,motionsuddenT], axis=1)

    # avg front distance
    avg_frontdist=np.mean(motionsDF[motionsDF['period_id']==period]['front_distance'])
    periodframe['avg_frontdist']= avg_frontdist
    
    
    # avg speed
    avg_speed=np.mean(motionsDF[motionsDF['period_id']==period]['speed'])
    periodframe['avg_speed']=avg_speed
    
    # avg collision 
    for i in milesDF.period_id.unique():
        if period==i:
            final_colmi=milesDF[milesDF['period_id']==period]['collision_mileage'].tail(1).iloc[0]
            num_coll_alert= len(milesDF[milesDF['period_id']==period]['collision_mileage'].unique())
            if num_coll_alert>0:
                mi_per_col=final_colmi/num_coll_alert
            else:
                mi_per_col=0
    periodframe['mi_per_col']=mi_per_col
    
    # avg bias
    avg_bias=np.mean(motionsDF[motionsDF['period_id']==period]['bias'])
    
    # collision reaction counts
    ColMotionids=motionsDF[(motionsDF['event']=='5')&(motionsDF['period_id']==period)]
    MotionCollision=pd.merge(ColMotionids,collisionsDF, how='inner', on='motion_id')['reaction'].value_counts().dropna().to_frame().reset_index()
    MotionCollision['index']= MotionCollision['index'].map(colReactd)
    for cEvent in colReactlist:
         if cEvent not in np.array(MotionCollision['index']):
                periodframe[cEvent]=0
    MotionCollision=MotionCollision.set_index('index')
    MotionCollisionT=MotionCollision.T
    periodframe=pd.concat([periodframe,MotionCollisionT], axis=1)
    

    # add periodDF to bigframe
    bigframe=pd.concat([bigframe, periodframe])

bigframe=bigframe.set_index('period_id')
bigframe.drop('suddenNaN', axis=1, level=None, inplace=True)
#bigframe.drop('colReactNaN', axis=1, level=None, inplace=True)
#bigframe.drop('suddenBrake', axis=1, level=None, inplace=True)

In [33]:
bigframe.head()

,CReact_changeL,CReact_ignore_departL,CReact_reduceSpeed,avg_frontdist,avg_speed,changeE,collisionE,departureE,frontE,mi_per_col,normalE,passingE,suddenAccel,suddenBrake,suddenE,suddenStop,total_mileage
period_id,,,,,,,,,,,,,,,,,
39,0,2,0,9.769231,28.692308,0,2,0,0,0.060424,1,0,10,0,10,0,0.734434
40,0,3,0,65.457143,50.942857,0,3,11,0,0.059055,13,0,41,0,43,0,11.015816
41,0,0,0,62.937500,26.625000,0,0,2,0,0.003986,1,0,13,0,13,0,0.485110
42,0,0,0,59.000000,37.200000,0,0,1,0,0.020828,1,0,3,0,3,0,1.075047
43,0,0,0,18.636364,31.242424,0,0,7,0,0.011047,3,0,21,0,23,1,1.750838
